In [1]:
from ckks_package.ckks import *

In [2]:
N = 2**15  # Ring degree
n = 2**4  # Number of slots
L_boot = 23  # Maximal level during bootstrapping
q0 = 2**29  # Smallest modulus
p = 2**23  # Scaling factor outside of bootstrapping
delta = q0  # Scaling factor during bootstrapping

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L_boot, q0, p, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.6) operations.

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([ 1.-3.j,  2.+2.j,  0.+0.j,  1.-3.j, -1.+0.j,  3.-1.j,  2.+2.j,
         2.-2.j,  3.-1.j,  0.+1.j, -1.+2.j, -2.+3.j, -3.-1.j,  3.-3.j,
        -0.-1.j, -1.+1.j]),
 array([-2.-2.j,  1.+2.j,  2.-1.j, -1.+3.j, -2.-2.j,  0.+1.j,  3.-3.j,
        -1.-1.j, -0.-3.j, -2.+0.j, -3.+2.j, -0.-2.j, -1.-2.j,  2.-3.j,
         2.-1.j,  2.+1.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[4718592 + 2959270X^1024 + 3709050X^2048 + 2403425X^3072 - 3694560X^4096 - 184337X^5120 + 6386559X^6144 + 2148182X^7168 - 2595093X^8192 - 2488841X^9216 - 3659113X^10240 - 4249655X^11264 - 200636X^12288 - 4759446X^13312 + 2234272X^14336 - 2251343X^15360 - 2097152X^16384 + 689778X^17408 - 165227X^18432 - 9721878X^19456 - 7549427X^20480 + 3008190X^21504 - 6942584X^22528 + 3335482X^23552 + 2595093X^24576 + 3830897X^25600 - 6858832X^26624 + 4725308X^27648 + 484379X^28672 + 1741407X^29696 + 169277X^30720 + 374268X^31744 mod(2^696),
 - 4810751X^1024 - 3066097X^2048 - 3600396X^3072 - 6248244X^4096 + 2304753X^5120 + 5714100X^6144 - 5290825X^7168 - 5560914X^8192 - 5019258X^9216 - 5654184X^10240 + 650556X^11264 - 4026985X^12288 + 2176150X^13312 + 4060585X^14336 + 11609517X^15360 - 5767168X^16384 - 12130X^17408 - 864088X^18432 - 3069712X^19456 - 1384289X^20480 + 1513367X^21504 - 2448742X^22528 - 1059207X^23552 - 4078004X^24576 + 1034485X^25600 - 2147522X^26624 + 670131X^27648 + 2871850X^28672 - 27

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^29 (level 29 out of 29).,
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^29 (level 29 out of 29).]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 9.99998931e-01-2.99999994e+00j  2.00000189e+00+1.99999811e+00j
 -7.04356879e-07-3.85345815e-06j  1.00000005e+00-3.00000201e+00j
 -1.00000123e+00-9.03714948e-07j  2.99999645e+00-1.00000137e+00j
  2.00000052e+00+1.99999915e+00j  2.00000038e+00-1.99999649e+00j
  2.99999661e+00-9.99997544e-01j -9.18215187e-07+1.00000107e+00j
 -1.00000030e+00+2.00000267e+00j -1.99999837e+00+3.00000160e+00j
 -2.99999974e+00-9.99998665e-01j  2.99999940e+00-2.99999856e+00j
  6.88791713e-08-9.99999009e-01j -1.00000258e+00+9.99999562e-01j]
[-2.00000071e+00-1.99999999e+00j  1.00000389e+00+1.99999890e+00j
  2.00000065e+00-9.99998589e-01j -9.99999054e-01+3.00000068e+00j
 -1.99999917e+00-1.99999907e+00j -5.78133222e-07+1.00000005e+00j
  3.00000194e+00-3.00000138e+00j -9.99996650e-01-1.00000153e+00j
  6.21464597e-07-2.99999957e+00j -1.99999906e+00+2.35256688e-06j
 -3.00000022e+00+2.00000150e+00j  1.18364707e-06-1.99999797e+00j
 -1.00000024e+00-1.99999751e+00j  1.99999949e+00-2.99999940e+00j
  2.00000098e+00-9.99999

In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^29 (level 29 out of 29).
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^28 (level 28 out of 29).


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-1.00000178e+00-4.99999992e+00j  3.00000579e+00+3.99999701e+00j
  1.99999995e+00-1.00000244e+00j  9.93985224e-07-1.33448763e-06j
 -3.00000040e+00-1.99999997e+00j  2.99999587e+00-1.31581937e-06j
  5.00000246e+00-1.00000223e+00j  1.00000373e+00-2.99999802e+00j
  2.99999723e+00-3.99999712e+00j -1.99999998e+00+1.00000343e+00j
 -4.00000052e+00+4.00000417e+00j -1.99999719e+00+1.00000362e+00j
 -3.99999998e+00-2.99999617e+00j  4.99999890e+00-5.99999796e+00j
  2.00000104e+00-1.99999856e+00j  9.99999603e-01+2.00000016e+00j]
[-7.99999818e+00+4.00000457e+00j -1.99998321e+00+6.00000591e+00j
 -4.90585077e-06-7.12662355e-06j  8.00001106e+00+6.00000082e+00j
  1.99999755e+00+2.00000200e+00j  9.99999988e-01+2.99999662e+00j
  1.20000056e+01-1.82619251e-06j -3.99999326e+00-1.43260616e-05j
 -2.99999055e+00-8.99999005e+00j -7.22309607e-07-2.00000143e+00j
 -1.00000729e+00-8.00001106e+00j  5.99999390e+00+3.99999592e+00j
  1.00000489e+00+6.99999099e+00j -2.99999751e+00-1.49999913e+01j
 -9.99999135e-01-1.99999

In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^0 (level 0 out of 29).
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^5 (level 5 out of 29).
[ 9.96862998e-01-2.99585190e+00j  1.99764188e+00+1.99849299e+00j
 -1.41897972e-03+4.83537302e-04j  1.00251720e+00-2.99730590e+00j
 -1.00140708e+00-6.55209389e-04j  2.99748311e+00-1.00204665e+00j
  1.99869069e+00+1.99672090e+00j  1.99853161e+00-1.99740033e+00j
  2.99729569e+00-1.00010720e+00j  2.07135160e-03+9.98210810e-01j
 -9.97566553e-01+1.99648493e+00j -1.99980121e+00+2.99829439e+00j
 -2.99657102e+00-9.99782932e-01j  2.99737863e+00-2.99644000e+00j
  8.63632901e-04-9.98377147e-01j -9.98580096e-01+9.98690335e-01j]
